### RDKit Conformer Browser
Derived from Greg Landrum's code by Malitha Humayun Kabir<br>
As a part of GSoC 2017 <br>
Project : RDKit - 3Dmol.js integration <br>
Mentors: Paul Czodrowski and Greg Landrum <br>
Date: 28th August 2017 <br>
Acknowledgement: Peter Gedeck reviewed and provided advice on restructuring codes and wrote MolViewState class.

##### This notebook assumes that  IPythonConsoleIntegration and LigandExtract merged in RDKit code base. 

In [1]:
from rdkit.Chem.Draw import IPythonConsoleIntegration as ipy
from rdkit.Chem import LigandExtract
from rdkit import Chem
from rdkit.Chem import AllChem
from six.moves import urllib

#### A detailed overview on how to use MolView3D has been provided at the following two links. Interested readers are referred to those. 

https://github.com/malithakabir/RDKitGSoC2017/blob/master/GSoC2017_notebook_1_ConformerBrowse_panels_and_confSelection.ipynb
https://github.com/malithakabir/RDKitGSoC2017/blob/master/GSoC2017_notebook_2_ConformerBrowse_with_proteins.ipynb

Checking whether 3DMol.js loaded.

In [2]:
ipy.Check3Dmolpromise()

Since the library is not loaded we need to run the following cell.

In [3]:
%%html
<script href="https://raw.githubusercontent.com/3dmol/3Dmol.js/master/release/3Dmol.js"></script>

Now we are creating list of molecules.

In [4]:
# Codes from Paolo Tosco (Slightly modified version)
url = 'https://github.com/rdkit/rdkit/raw/master/Docs/Book/data/cdk2.sdf'
response = urllib.request.urlopen(url)
data = response.read()
suppl = Chem.SDMolSupplier()
suppl.SetData(data)
molList = [mol for mol in suppl]

In [5]:
print(len(molList))

47


For the convenience of demonstration, we will be using only 4 molecules from the list.

In [6]:
molList_2 = molList[:4]

Now it is time for generating few conformers per molecule. In this example we are generating 10 conformers for each molecule.

In [7]:
params = AllChem.ETKDG()
params.numThreads=3
for m in molList_2:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)


Converting molList (list type object) to dictionary (type = dict)

In [8]:
molList_3 = [(str(ids),mol) for ids,mol in enumerate(molList_2)]
molDict = dict(molList_3)
print('molecule keys within dictionary is : ' + str(type(list(molDict.keys())[0])))
print('The following are the molecule keys : ')
print(list(molDict.keys()))

molecule keys within dictionary is : <type 'str'>
The following are the molecule keys : 
['1', '0', '3', '2']


In [9]:
molDict

{'0': <rdkit.Chem.rdchem.Mol at 0x7f617d625210>,
 '1': <rdkit.Chem.rdchem.Mol at 0x7f617d625280>,
 '2': <rdkit.Chem.rdchem.Mol at 0x7f617d6252f0>,
 '3': <rdkit.Chem.rdchem.Mol at 0x7f617d625360>}

Processing molDict to make it compatible for MolView3D can be done by ProcessLigandDict function. This function adds a tag 'parent' to each of the molecule present. We can change this tag. But changing this tag should be tracked down. Or we can accept default. 

In [10]:
kpm = 'newparent'
molDict_2 = ipy.ProcessLigandDict(molDict, keyForParentMol = kpm)

In [11]:
molDict_2

{'0': {'newparent': <rdkit.Chem.rdchem.Mol at 0x7f617d625210>},
 '1': {'newparent': <rdkit.Chem.rdchem.Mol at 0x7f617d625280>},
 '2': {'newparent': <rdkit.Chem.rdchem.Mol at 0x7f617d6252f0>},
 '3': {'newparent': <rdkit.Chem.rdchem.Mol at 0x7f617d625360>}}

Great! we have got new tags in molDict_2. Now we are ready for energy minimization.

MinimizeLigand function can be used to perform energy minimization of ligand. If we accept default then we just need to parse only one argument (ligandDict = molDict_2).<br><br> Here are the default arguments for MinimizeLigand function except ligandDict: <br>keyForParentMol = 'parent'<br>keyForMinimizedMol = 'minimized'<br>energyDataKey = 'energy'<br>molAndConfIds = 'allConfs'<br>ff = 'UFF'<br>maxIters = 50 <br><br>And we are trying to override those arguments so that we can use the same dictionary again and again.


In [12]:
kmm = 'newminimized'
ke = 'minimizationEnergy'
ids = 'allConfs'
forceField = 'UFF'
maximumIterations = 100

In [13]:
molDict_3 = ipy.MinimizeLigand(ligandDict = molDict_2, 
                               keyForParentMol = kpm, 
                               keyForMinimizedMol = kmm,
                               energyDataKey = ke,
                               molAndConfIds = ids, 
                               ff = forceField, 
                               maxIters = maximumIterations)

In [14]:
molDict_3

{'0': {'minimizationEnergy': {0: 54.37722035959141,
   1: 52.870120130847205,
   2: 54.628867021683185,
   3: 55.39062883633577,
   4: 54.018535072057865,
   5: 54.375169529453196,
   6: 54.385585955187,
   7: 54.628867023023076,
   8: 54.394363913159374,
   9: 54.37538343655261},
  'newminimized': <rdkit.Chem.rdchem.Mol at 0x7f617d62a910>,
  'newparent': <rdkit.Chem.rdchem.Mol at 0x7f617d625210>},
 '1': {'minimizationEnergy': {0: 59.877198010058564,
   1: 59.61535881452488,
   2: 60.571271828302024,
   3: 60.188943233746336,
   4: 60.57127184583709,
   5: 59.62132531962908,
   6: 59.87765851866043,
   7: 59.615358813127706,
   8: 60.73622788924266,
   9: 59.61535880769493},
  'newminimized': <rdkit.Chem.rdchem.Mol at 0x7f617d62a830>,
  'newparent': <rdkit.Chem.rdchem.Mol at 0x7f617d625280>},
 '2': {'minimizationEnergy': {0: 64.33622381619975,
   1: 64.93001602412583,
   2: 64.20217153489754,
   3: 64.76374578578816,
   4: 64.76374583029842,
   5: 64.33622383934015,
   6: 65.7038246220

Now we have everything set for viewing minimization energy data with conformers. <br><br> Please note that values for the arguments ligStyle and emLigStyle are already in default settings. <br> <br> And since we have override the default tags of ligDict we need to input new tags in MolView3D also. 

In [15]:
view = ipy.MolView3D(ligandDict = molDict_3, 
                     ligSelPanel = 'full',
                     ligStyle  = 'line', 
                     emLigStyle  = 'line', 
                     stylePanel = 'emlig',
                     emPanel = True,
                     labelPanel = True,
                     keyForParentMol = kpm, 
                     keyForMinimizedMol = kmm,
                     energyDataKey = ke)

In [17]:
view.SelectMolAndConf(molAndConfIds = (('1',7),))

### Thank you very much for your time. Have a great day! 